#### 11.1.2 进程

In [17]:
import multiprocessing as mp

def washer(dishes, output):
    for dish in dishes:
        print('Washing', dish, 'dish')
        output.put(dish)
        
def dryer(input):
    while True:
        dish = input.get()
        print('Drying', dish, 'dish')
        input.task_done()
        
dish_queue = mp.JoinableQueue()
dryer_proc = mp.Process(target=dryer, args=(dish_queue,))
dryer_proc.daemon = True
dryer_proc.start()

dishes = ['salad', 'bread', 'entree', 'dessert']
washer(dishes, dish_queue)
dish_queue.join()

Drying salad dish
Drying bread dish
Drying entree dish
Drying dessert dish
Washing salad dish
Washing bread dish
Washing entree dish
Washing dessert dish


#### 11.1.3 进程

In [19]:
import threading, queue
import time

def washer(dishes, dish_queue):
    for dish in dishes:
        print("Washing", dish)
        time.sleep(5)
        dish_queue.put(dish)
        
def dryer(dish_queue):
    while True:
        dish = dish_queue.get()
        print("Drying", dish)
        time.sleep(10)
        dish_queue.task_done()

dish_queue = queue.Queue()
for n in range(2):
    dryer_thread = threading.Thread(target=dryer, args=(dish_queue, ))
    dryer_thread.start()
    
dishes = ['salad', 'bread', 'entree', 'dessert']
washer(dishes, dish_queue)
dish_queue.join()

Washing salad
WashingDrying bread
 salad
Washing entree
Drying bread
WashingDrying dessert
 entree
Drying dessert


#### 11.1.4 绿色线程和gevent

In [ ]:
## save as gevent_test.py
import gevent
from gevent import socket
hosts=["www.github.com", "www.amazon.com"]
jobs = [gevent.spawn(gevent.socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout=5)
for job in jobs:
    print(job.value)

In [ ]:
## save as gevent_monkey.py
import gevent
from gevent import monkey; monkey.patch_all()
import socket
hosts=["www.github.com", "www.amazon.com"]
jobs = [gevent.spawn(socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout=5)
for job in jobs:
    print(job.value)

#### 11.1.5 twisted

In [ ]:
## save as knock_server.py

# -*- coding: utf-8 -*-

from twisted.internet import protocol, reactor

class Knock(protocol.Protocol):
    def dataReceived(self, data):
        print("Client:", data)
        if data.startswith("Knock knock".encode('ascii')):
            response = "Who's there?"
        else:
            response = str(data) + " who?"
        print("Server:", response)
        self.transport.write(response.encode('ascii'))
        
class KnockFactory(protocol.Factory):
    def buildProtocol(self, addr):
        return Knock()

reactor.listenTCP(8000, KnockFactory())
reactor.run()

In [2]:
## save as knock_client.py

# -*- coding: utf-8 -*-

from twisted.internet import reactor, protocol

class KnockClient(protocol.Protocol):
    def connectionMade(self):
        self.transport.write("Knock knock".encode('ascii'))
        
    def dataReceived(self, data):
        if data.startswith("Who's there?".encode('ascii')):
            response = "Disappearing client"
            self.transport.write(response.encode('ascii'))
        else:
            self.transport.loseConnection()
            reactor.stop()
class KnockFactory(protocol.ClientFactory):
    protocol = KnockClient
    
def main():
    f = KnockFactory()
    reactor.connectTCP("localhost", 8000, f)
    reactor.run()
    
if __name__ == '__main__':
    main()

ReactorNotRestartable: 

#### 11.2.4 套接字

In [ ]:
# udp_server.py
from datetime import datetime
import socket

server_address = ('localhost', 6789)
max_size = 4096

print('Starting the server at', datetime.now())
print('Waiting for a client to call.')
server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.bind(server_address)

data, client = server.recvfrom(max_size)

print('At', datetime.now(), client, 'said', data)
server.sendto(b'Are you talking to me?', client)
server.close()

In [3]:
# udp_client.py
import socket
from datetime import datetime

server_address = ('localhost', 6789)
max_size = 4096

print('Starting the client at', datetime.now())
client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.sendto(b'Hey!', server_address)
data, server = client.recvfrom(max_size)
print('At', datetime.now(), server, 'said', data)
client.close()

Starting the client at 2018-05-29 20:35:12.343803


KeyboardInterrupt: 

按顺序启动udp_server.py、udp_client.py，得到如下结果：

```
$ python udp_server.py 
Starting the server at 2018-05-29 20:36:36.007946
Waiting for a client to call.
At 2018-05-29 20:36:52.606674 ('127.0.0.1', 50087) said b'Hey!'

$ python udp_client.py 
Starting the client at 2018-05-29 20:36:52.605319
At 2018-05-29 20:36:52.606749 ('127.0.0.1', 6789) said b'Are you talking to me?'
```

In [4]:
# tcp_server.py

from datetime import datetime
import socket

address = ('localhost', 6789)
max_size = 1000

print('Starting the server at', datetime.now())
print('Waiting for a client to call.')
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(address)
server.listen(5)

client, addr = server.accept()
data = client.recv(max_size)

print('At', datetime.now(), client, 'said', data)
client.sendall(b'Are you talking to me?')
client.close()
server.close()

Starting the server at 2018-05-29 21:52:32.257351
Waiting for a client to call.


KeyboardInterrupt: 

In [ ]:
# tcp_client.py

import socket
from datetime import datetime

address = ('localhost', 6789)
max_size = 1000

print('Starting the client at', datetime.now())
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(address)
client.sendall(b'Hey!')
data = client.recv(max_size)
print('At', datetime.now(), 'someone replied', data)
client.close()

按顺序启动tcp_server.py、tcp_client.py，得到如下结果：

```
$ python tcp_server.py 
Starting the server at 2018-05-29 21:50:23.316354
Waiting for a client to call.
At 2018-05-29 21:50:26.358545 <socket.socket fd=6, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 6789), raddr=('127.0.0.1', 51440)> said b'Hey!'

$ python tcp_client.py
Starting the client at 2018-05-29 21:50:26.356516
At 2018-05-29 21:50:26.358683 someone replied b'Are you talking to me?'
```

#### 11.2.7 网络服务

1. 域名系统

In [11]:
import socket

# 返回主机IP地址
host = socket.gethostbyname('www.github.com')
print("host=", host)

# 返回名称、一个可选名称列表和一个地址列表
host_ex = socket.gethostbyname_ex('www.github.com')
print("host_ex=", host_ex) 

# 查询可以用于创建套接字的IP和端口
addrinfo = socket.getaddrinfo('www.github.com', 80)
print("addrinfo=", addrinfo)

addrinfo_udp = socket.getaddrinfo('www.github.com', 80, socket.AF_INET, socket.SOCK_DGRAM)
print("addrinfo_udp=", addrinfo_udp)

addrinfo_tcp = socket.getaddrinfo('www.github.com', 80, socket.AF_INET, socket.SOCK_STREAM)
print("addrinfo_tcp=", addrinfo_tcp)

# IANA为特定服务的保留端口号查询
port = socket.getservbyname('http')
name = socket.getservbyport(80)
print(name, port)

port = socket.getservbyname('https')
name = socket.getservbyport(443)
print(name, port)

port = socket.getservbyname('telnet')
name = socket.getservbyport(23)
print(name, port)

host= 13.229.188.59
host_ex= ('github.com', ['www.github.com'], ['13.229.188.59', '52.74.223.119', '13.250.177.223'])
addrinfo= [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('13.229.188.59', 80)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('13.229.188.59', 80)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('52.74.223.119', 80)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('52.74.223.119', 80)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('13.250.177.223', 80)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('13.250.177.223', 80))]
addrinfo_udp= [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('13.229.188.59', 80)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('52.74.223.119', 80)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('13.250.177.223', 80))]
addrinfo_tcp= [(<AddressFamily.AF_INET: 2>, <SocketK